#### Dans un premier temps il faut aller sur le site IMDb et trouver tous les films pour l'année 2017 et les classé par vote

#### Ensuite il faut analyser l'url du site

#### Bien analyser cette partie de l'url : release_date=2017-01-01,2017-12-31&sort=num_votes,desc

In [1]:
# On va importé la fonction get() du module requests
# Assigner l'url de la page à la variable url
# Télécharger la page et assigner le résultat à la variable response
# Afficher un extrait du résultat

In [23]:
from requests import get
url = 'https://www.imdb.com/search/title?release_date=2017-01-01,2017-12-31&sort=num_votes,desc&ref_=adv_prv'
response = get(url)
print(response.text[:500])



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle",


## Structure HTML de la page

In [5]:
# Bien inspecter la page pour trouver la class qui correpond au 50 films par page
# importer la classe BeautifulSoup du package bs4
# Extraire le code HTML
# Utiliser la méthode find.all() pour extraire les éléments souhaités

In [24]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [25]:
movie_containers = html_soup.find_all('div', class_="lister-item mode-advanced")
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [26]:
# Extraire l'année de sortie du premier film
# Extraire la note IMDB (à convertir en nombre à virgule)
# Extraire la note Metacritic (à convertir en entier)
# Extraire le nombre de votes (utiliser paramètre attrs)

In [27]:
first_movie = movie_containers[0]
first_movie

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>
<p class="text-muted">
<span class="certificate">12</span>
<span class="ghost">|</span>
<span class="runtime">137 min</span>
<span class="ghost">|</span>
<span class="genre">
Ac

On va tout d'abord extraire le titre du premier film / inspecter le titre

In [28]:
# titre du film 

first_movie.div


<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt3315342"></div>
</div>

In [29]:
first_movie.a # on cherche sur les sélecteurs ce qui peux correspondre

<a href="/title/tt3315342/"> <img alt="Logan" class="loadlate" data-tconst="tt3315342" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYzc5MTU4N2EtYTkyMi00NjdhLTg3NWEtMTY4OTEyMzJhZTAzXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB470041630_.png" width="67"/>
</a>

In [30]:
first_movie.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt3315342/">Logan</a>
<span class="lister-item-year text-muted unbold">(2017)</span>
</h3>

In [31]:
first_name = first_movie.h3.a.text
first_name

'Logan'

In [36]:
# Année de sortie
first_year = first_movie.h3.find('span', class_="lister-item-year text-muted unbold")

In [37]:
first_year = first_year.text
first_year

'(2017)'

In [38]:
# Note IMDb (inspecter note pour la balise)
first_movie.strong

<strong>8.1</strong>

In [39]:
first_imdb = float(first_movie.strong.text) # convertir en float (nombre à virgule)
first_imdb 

8.1

In [41]:
# Note Metacritic (inspecter balise )
first_metascore = first_movie.find('span', class_='metascore favorable')
first_metascore = int(first_metascore.text) # convertir en un nombre entier
first_metascore

77

In [42]:
# Nombre de vote (inspecter la balise)
first_votes = first_movie.find('span', attrs = {'name': 'nv'})
first_votes

<span data-value="553357" name="nv">553,357</span>

In [43]:
first_votes['data-value']

'553357'

In [44]:
first_votes = int(first_votes['data-value'])
first_votes

553357

## Script pour scaper une seule page ( et non éléments par éléments )

In [54]:
# On crée des listes vides pour toutes nos informations
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# on reprend notre data movie_containers pour y extraire l'information
for container in movie_containers:
    
    # si le film a une note Metacritic, on l'extrait
    if container.find('div', class_='ratings-metascore') is not None:
        
        # le titre du film
        name = container.h3.a.text
        names.append(name) # on ajoute chaque élément name de la boucle à la liste names
        
        # L'année de sortie
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # la note IMDB
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Metascore
        m_score = container.find('span', class_='metascore').text
        metascores.append(int(m_score))
        
        # le nombre de votes
        vote = container.find('span', attrs = {'name':'nv'})['data-value']
        votes.append(int(vote))

In [56]:
print(names)

['Logan', 'Wonder Woman', 'Les gardiens de la galaxie Vol. 2', 'Thor Ragnarok', 'Dunkerque', 'Star Wars: Episode VIII - Les derniers Jedi', 'Spider-Man: Homecoming', 'Get Out', 'Blade Runner 2049', 'Baby Driver', 'Ça', '3 Billboards: Les panneaux de la vengeance', 'Justice League', "La forme de l'eau", 'John Wick 2', 'Coco', 'Jumanji: Bienvenue dans la jungle', 'La Belle et la Bête', 'Kong: Skull Island', "Kingsman: Le cercle d'or", 'Pirates des Caraïbes: la Vengeance de Salazar', 'Alien: Covenant', 'La planète des singes: Suprématie', 'The Greatest Showman', 'Life: Origine inconnue', 'Fast & Furious 8', "Le crime de l'Orient-Express", 'Lady Bird', 'Ghost in the Shell', "Le roi Arthur: La légende d'Excalibur", 'Wind River', 'Hitman & Bodyguard', 'Mother', 'La momie', 'Call Me by Your Name', 'Atomic Blonde', 'Bright', 'Moi, Tonya', 'Valérian et la Cité des Mille Planètes', 'Baywatch: Alerte à Malibu', 'Les heures sombres', 'Barry Seal: American Traffic', 'Transformers: The Last Knight',

In [58]:
print(years)

['(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(I) (2017)', '(2017)', '(2017)', '(I) (2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(I) (2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(I) (2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(I) (2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)', '(2017)']


In [60]:
print(imdb_ratings)

[8.1, 7.5, 7.7, 7.9, 7.9, 7.2, 7.5, 7.7, 8.0, 7.6, 7.4, 8.2, 6.5, 7.3, 7.5, 8.4, 7.0, 7.2, 6.6, 6.8, 6.6, 6.4, 7.5, 7.6, 6.6, 6.7, 6.5, 7.4, 6.4, 6.8, 7.7, 6.9, 6.6, 5.5, 7.9, 6.7, 6.4, 7.5, 6.5, 5.6, 7.4, 7.2, 5.2, 7.3, 7.4]


In [61]:
print(metascores)

[77, 76, 67, 74, 94, 85, 73, 84, 81, 86, 69, 88, 45, 87, 75, 81, 58, 65, 62, 44, 39, 65, 82, 48, 54, 56, 52, 94, 52, 41, 73, 47, 75, 34, 93, 63, 29, 77, 51, 37, 75, 65, 27, 75, 76]


In [62]:
print(votes)

[553357, 483541, 475864, 472821, 462265, 459289, 418271, 394110, 373188, 358573, 353322, 339449, 326220, 304513, 273039, 271954, 239926, 235837, 234261, 227742, 226588, 225220, 199904, 197808, 181750, 178289, 177214, 175653, 175390, 167729, 167433, 160362, 154944, 152909, 151880, 148825, 146881, 139671, 138611, 135506, 135179, 129536, 117774, 115510, 110720]


In [63]:
print(len(names))

45


## Affichage en DataFrame avec pandas

In [64]:
import pandas as pd
test_df = pd.DataFrame({
    'movie': names,
    'year': years,
    'imdb': imdb_ratings,
    'metascore': metascores,
    'votes': votes
})

In [66]:
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 5 columns):
movie        45 non-null object
year         45 non-null object
imdb         45 non-null float64
metascore    45 non-null int64
votes        45 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ KB
None


In [67]:
test_df

,movie,year,imdb,metascore,votes
0,Logan,(2017),8.1,77,553357
1,Wonder Woman,(2017),7.5,76,483541
2,Les gardiens de la galaxie Vol. 2,(2017),7.7,67,475864
3,Thor Ragnarok,(2017),7.9,74,472821
4,Dunkerque,(2017),7.9,94,462265
5,Star Wars: Episode VIII - Les derniers Jedi,(2017),7.2,85,459289
6,Spider-Man: Homecoming,(2017),7.5,73,418271
7,Get Out,(I) (2017),7.7,84,394110
8,Blade Runner 2049,(2017),8.0,81,373188
9,Baby Driver,(2017),7.6,86,358573


## Script pour toutes les pages

In [68]:
pages = [str(i) for i in range(1,5)]
pages

['1', '2', '3', '4']

In [69]:
years_url = [str(i) for i in range(2000,2018)]
years_url

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017']

## Contrôler le taux de requêtes envoyées (pour ne pas être banni)

In [70]:
from time import sleep
from random import randint

for a in range(0,5):
    print('booh')
    sleep(randint(1,4))

booh
booh
booh
booh
booh


In [71]:
# On va créer un script qui permet de mesurer la fréquence de requête , le nombre de requête et le code status (réussite ou echec)

In [73]:
from time import time
from IPython.display import clear_output

start_time = time() # on fixe le temps de début
requests = 0 # la variable requests va compter le nombre de requêtes
for _ in range(5):
    requests +=1 # on incrémente la variable requests d'une unité a chaque passage de boucle
    sleep(randint(1,3)) # on fait une pause de 1 à 3 secondes
    elapsed_time = time() -start_time # on calcule le temps écoulé depuis la première requête
    print('Request: {}, Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5, Frequency: 0.41646617699725147 requests/s


In [75]:
from warnings import warn
warn('Attention')

C:\Users\Joan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Attention
  


## Le Script final

In [79]:
# 1. Redéclarer les listes
names = []
years = []
imdb_ratings = []
metascores = []
votes = []

# 2. Préparer l'affichage des boucles
start_time = time()
requests = 0

# 3. Pour chaque année de l'intervalle 2000-2017
for year_url in years_url:
    
    # 4. boucle pour chaque page entre 1 et 4
    for page in pages:
        
        # 5. faire une requête GET
        response = get('https://www.imdb.com/search/title?release_date='+ year_url +'&sort=num_votes,desc&page='+ page)
        
        # 6. Pause sur la boucle de 8 à 15 secondes
        sleep(randint(8,15))
        
        # 7. afficher les informations sur les requêtes
        requests +=1
        elapsed_time = time() - start_time
        print('Request: {}; Frequence: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)
        
        # 8. Avertir sur le code status est différent de 200
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
        # 9. Stopper la boucle si le nombre de requête est supérieur à 72
        if requests > 72:
            warn('Nombre de requêtes trop important')
            breack
        
        # 10. Extraire le HTML avec BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')
        
        # 11. Sélectionner les 50 films de chaque page (container)
        mv_containers = page_html.find_all('div', class_="lister-item mode-advanced")
        
        # 12. Boucle pour chaque container
        for container in mv_containers:
            # 13. si le film a un Metascore
            if container.find('div', class_='ratings-metascore') is not None:
                              
                              # 14. scrape le titre
                              name = container.h3.a.text
                              names.append(name)
                              
                              # 15. scrape l'année
                              year = container.h3.find('span', class_='lister-item-year').text
                              years.append(year)
                              
                              # 16. scrape la note IMDB
                              imdb = float(container.strong.text)
                              imdb_ratings.append(imdb)
                              
                              # 17. scrape le Metascore
                              metascore = container.find('span', class_='metascore').text
                              metascores.append(int(metascore))
                              
                              # 18. scrape le nombre de votes
                              vote = container.find('span', attrs = {'name':'nv'})['data-value']
                              

Request: 72; Frequence: 0.07823390875293072 requests/s


In [84]:
names

['Gladiator',
 'Memento',
 'Snatch - Tu braques ou tu raques',
 'Requiem for a Dream',
 'X-Men',
 'Seul au monde',
 'American Psycho',
 'Incassable',
 'Mon beau-père et moi',
 'M-I:2 Mission: Impossible 2',
 "O' Brother",
 '60 secondes chrono',
 'Tigre et dragon',
 'The patriot: Le chemin de la liberté',
 'Presque célèbre',
 'Scary Movie',
 'Pitch Black',
 'Destination finale',
 "Fou(s) d'Irène",
 'La plage',
 'Amours chiennes',
 'Traffic',
 'Le plus beau des combats',
 'Le Grinch',
 'Ce que veulent les femmes',
 'Miss Détective',
 'Le chocolat',
 'Chicken Run',
 'Charlie et ses drôles de dames',
 "Kuzco, l'empereur mégalo",
 'Battle Royale',
 'Erin Brockovich, seule contre tous',
 'High Fidelity',
 'Road Trip',
 'En pleine tempête',
 'Eh mec! Elle est où ma caisse?',
 'Billy Elliot',
 "Hollow man - L'homme sans ombre",
 'Scream 3',
 'Shanghai Kid',
 "À l'aube du 6ème jour",
 'Apparences',
 'Un monde meilleur',
 'In the Mood for Love',
 'Mon voisin le tueur',
 'Coyote Girls',
 'Les che

## Affichage final de la data

In [ ]:
import pandas as pd
movie_ratings = pd.DataFrame({
    'movie': names,
    'year': years,
    'imdb': imdb_ratings,
    'metascore': metascores,
    'votes': votes
})